In [54]:
from datasets import load_dataset
import pandas as pd
ds = load_dataset("HuggingFaceH4/aime_2024")
df = pd.DataFrame(ds['train'])

README.md:   0%|          | 0.00/932 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/81.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/30 [00:00<?, ? examples/s]

In [56]:
languages=[
    "Korean",
    "Afrikaans",
    "Arabic",
    "Chinese (Simplified)",
    "French",
    "English",
    "German",
    "Hebrew",
    "Indonesian",
    "Vietnamese",
    "Italian",
    "Japanese",
    "Spanish",
    "Turkish",
]

In [67]:
from prompts import question_translation_template
from litellm import batch_completion
from tqdm import tqdm
data = []
for lang in tqdm(languages):
    qrys = []
    for _,row in df.iterrows():
        query = question_translation_template.replace('<source_question>',row.problem)
        query = query.replace('<language>',lang)
        qrys.append(
            [
                {"role": "system", "content": "You are a skilled translator."},
                {'role':'user','content':query}
            ]
        )
    responses = batch_completion(
        model="gpt-4o",
        messages = qrys,
        temperature=0.0
    )
    outputs = [res.choices[0].message.content for res in responses]
    df[f'problem_{lang}'] = outputs

100%|██████████| 14/14 [01:31<00:00,  6.55s/it]


In [71]:
def extract_translation(text):
    match = re.search(r'<translation>(.*?)</translation>', text, re.DOTALL)
    return match.group(1).strip() if match else None

In [75]:
cols =  ['problem_Korean',
       'problem_Afrikaans', 'problem_Arabic', 'problem_Chinese (Simplified)',
       'problem_French', 'problem_English', 'problem_German', 'problem_Hebrew',
       'problem_Indonesian', 'problem_Vietnamese', 'problem_Italian',
       'problem_Japanese', 'problem_Spanish', 'problem_Turkish']
for c in cols:
    df[c] = df[c].apply(lambda x:extract_translation(x))

In [79]:
from datasets import Dataset

ds = Dataset.from_pandas(df)
ds.push_to_hub('amphora/m-aime-2024')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/amphora/m-aime-2024/commit/2808f8e91371b616c423135f06566ae1eb990b02', commit_message='Upload dataset', commit_description='', oid='2808f8e91371b616c423135f06566ae1eb990b02', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/amphora/m-aime-2024', endpoint='https://huggingface.co', repo_type='dataset', repo_id='amphora/m-aime-2024'), pr_revision=None, pr_num=None)